In [291]:
#
# En este dataset se desea pronosticar el precio de vhiculos usados. El dataset
# original contiene las siguientes columnas:
#
# - Car_Name: Nombre del vehiculo.
# - Year: Año de fabricación.
# - Selling_Price: Precio de venta.
# - Present_Price: Precio actual.
# - Driven_Kms: Kilometraje recorrido.
# - Fuel_type: Tipo de combustible.
# - Selling_Type: Tipo de vendedor.
# - Transmission: Tipo de transmisión.
# - Owner: Número de propietarios.
#
# El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
# en la carpeta "files/input/".
#
# Los pasos que debe seguir para la construcción de un modelo de
# pronostico están descritos a continuación.
#
#
# Paso 1.
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.
#
#
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.
#
#
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.
#
#
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.
#
#
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.
#
#
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

In [292]:
import os
import json
import zipfile
import gzip
import pickle
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import median_absolute_error, mean_squared_error, r2_score

In [293]:
def clean_data(df):
    df = df.copy()
    df['Age'] = 2021 - df['Year']
    df.drop(columns=['Year', 'Car_Name'], inplace=True)
    df.dropna(inplace=True)
    return df

In [294]:
def load_data_train():
    with zipfile.ZipFile('../files/input/train_data.csv.zip', 'r') as file:
        print(file.namelist())
        with file.open('train_data.csv') as f:
            data = pd.read_csv(f)
    return data

def load_data_test():
    with zipfile.ZipFile('../files/input/test_data.csv.zip', 'r') as file:
        print(file.namelist())
        with file.open('test_data.csv') as f:
            data = pd.read_csv(f)
    return data

data_train = load_data_train()
data_test = load_data_test()
data_train = clean_data(data_train)
data_test = clean_data(data_test)
data_train.head()

['train_data.csv']
['test_data.csv']


,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,7.40,8.500,15059,Petrol,Dealer,Automatic,0,5
1,4.00,4.600,30000,Petrol,Dealer,Manual,0,8
2,0.50,0.826,6000,Petrol,Individual,Manual,0,10
3,3.15,4.430,15000,Petrol,Dealer,Manual,0,5
4,1.25,1.500,15000,Petrol,Individual,Manual,0,8


In [295]:
def split_data(data):
    x = data.drop(columns='Present_Price')
    y = data['Present_Price']
    return x, y

x_train, y_train = split_data(data_train)
x_test, y_test = split_data(data_test)

In [296]:
def create_pipeline():
    categorical_features = ['Fuel_Type', 'Selling_type', 'Transmission']
    numerical_features = list(set(x_train.columns) - set(categorical_features))

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", MinMaxScaler(), numerical_features),
            ("cat", OneHotEncoder(), categorical_features)
        ],
        remainder="passthrough"
    )

    k_best = SelectKBest(f_regression, k='all')

    model = LinearRegression()

    model_pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("k_best", k_best),
            ("model", model)
        ]
    )

    return model_pipeline

model_pipeline = create_pipeline()

In [297]:
def optimize_hyperparameters(model, x, y):
    param_grid = {
	"k_best__k": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
	"model__fit_intercept": [True, False]
}

    grid_search = GridSearchCV(
        model,
        param_grid=param_grid,
        cv=10,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
        refit=True,
        verbose=1
    )

    grid_search.fit(x, y)

    print(grid_search.best_params_)

    return grid_search

grid_search = optimize_hyperparameters(model_pipeline, x_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits
{'k_best__k': 11, 'model__fit_intercept': False}


In [298]:
def save_model(model):
    os.makedirs('../files/models', exist_ok=True)
    with gzip.open('../files/models/model.pkl.gz', 'wb') as file:
        pickle.dump(model, file)

save_model(grid_search)

In [299]:
y_train_pred = grid_search.predict(x_train)
y_test_pred = grid_search.predict(x_test)

def metrics_calc(y_true, y_pred, dataset):
        return {
            'type': 'metrics',
            'dataset': dataset,
            'r2': r2_score(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'mad': median_absolute_error(y_true, y_pred)
        }

metrics = [
    metrics_calc(y_train, y_train_pred, 'train'),
    metrics_calc(y_test, y_test_pred, 'test'),
]
os.makedirs("../files/output", exist_ok=True)
with open("../files/output/metrics.json", "w") as f:
    for metric in metrics:
        f.write(json.dumps(metric) + "\n")